In [2]:
import pandas as pd
from datetime import timedelta
pd.set_option("display.float_format", "{:.2f}".format)

In [ ]:
# Load dataset
file_path = ("data/sample_accounts.parquet")
df = pd.read_parquet(file_path)
df.sort_values(by=['subscriber_count'])


,account_id,subscriber_count,date,index,categories
145867,f769a9d5-1d83-444a-8e76-393d5001c605,1000.14,2023-06-21T23:59:39,6803,Gaming;Investment
158778,cbc9c2d3-cf6a-411b-bfaa-2435ceabdc42,1000.14,2023-06-15T20:48:01,206,Gaming
403382,6fd661b4-3b0e-41c2-b8f3-742aa1e19d10,1000.24,2024-04-02T22:04:57,4755,Podcast;Investment
242055,1224ba4c-ff24-482a-82bf-1c720af873ba,1000.25,2023-03-07T22:38:28,2539,Gaming
441816,87902167-52fb-4d18-87e6-05cee479b567,1000.27,2023-09-30T11:24:25,6025,Investment
...,...,...,...,...,...
439081,450aad65-2fcc-403d-a744-bce5b208c18a,3735555916183.91,2023-04-01T20:27:14,3849,Podcast
437942,450aad65-2fcc-403d-a744-bce5b208c18a,4003866129266.53,2023-08-15T21:16:48,3849,Podcast
435151,450aad65-2fcc-403d-a744-bce5b208c18a,4118970122693.49,2023-04-29T05:31:37,3849,Podcast
436638,450aad65-2fcc-403d-a744-bce5b208c18a,4632419793176.07,2024-04-19T17:29:55,3849,Podcast


In [31]:
# Transform dataset
df_transformed = df.copy()
df_transformed["date"] = pd.to_datetime(df_transformed["date"])  # Ensure consistent timestamp format
df_transformed = df_transformed[df_transformed["subscriber_count"] > 1000]  # Filter accounts with >1000 subscribers
df_transformed["categories"] = df_transformed["categories"].str.split(";")  # Split categories by ";"

In [35]:
df_transformed

,account_id,subscriber_count,date,index,categories
0,342cc0d0-fe2f-486c-8015-2ef70713054f,615572546.00,2024-02-04 02:51:29,0,[Investment]
1,a2cbe3ce-8bb7-4192-a247-352bd24d305a,197443174.00,2024-02-04 02:51:29,1,"[Gaming, Investment]"
2,c5ebe195-4e38-40a3-8fb2-84417d4902c0,584374838.00,2024-02-04 02:51:29,2,[Investment]
3,c0172ad3-07d2-421e-a184-b05624f6530d,871923701.00,2024-02-04 02:51:29,3,[Gaming]
4,25bec5e8-d626-443b-a414-2bc835a21ecb,468931616.00,2024-02-04 02:51:29,4,[Investment]
...,...,...,...,...,...
444717,9afe7eb4-683b-4971-8cd8-34d4d35f6390,16515.95,2023-11-30 11:47:14,9589,"[Podcast, Gaming]"
444718,57132976-f713-4fba-87f3-cd003b5eb581,4404.68,2023-11-30 11:47:14,9630,"[Podcast, Education, Investment]"
444719,b0ba7cd4-2736-4dff-bd54-dce17aab587f,2680.45,2023-11-30 11:47:14,9702,[Investment]
444720,c0572cbb-1768-4313-a565-9a4a4634e055,1168283430.97,2023-11-30 11:47:14,9768,[Investment]


In [29]:
# Define analysis window (30 days or 1 month)
analysis_window = df_transformed["date"].max() - timedelta(days=30) # Define the analysis period (last 30 days from the most recent date in the dataset)
recent_entries = df_transformed[df_transformed["date"] > analysis_window] # Filter entries to include only those within the analysis window

In [30]:
# Sort and get first and last subscriber count
sorted_records = recent_entries.sort_values(by=["account_id", "date"]) # Sort records by account_id and date to maintain chronological order
initial_snapshot = sorted_records.drop_duplicates(subset=["account_id"], keep="first")[["account_id", "subscriber_count"]].rename(columns={"subscriber_count": "baseline_subscribers"}) # Get the first recorded subscriber count for each account in the period
latest_snapshot = sorted_records.drop_duplicates(subset=["account_id"], keep="last")[["account_id", "subscriber_count"]].rename(columns={"subscriber_count": "current_subscribers"}) # Get the last recorded subscriber count for each account in the period

In [ ]:
# Merge snapshots
trend_data = initial_snapshot.merge(latest_snapshot, on="account_id", how="inner") # Merge the first and last subscriber counts to analyze growth trend
trend_data["growth_values"] = trend_data["current_subscribers"] - trend_data["baseline_subscribers"] # Calculate absolute growth in subscriber count
trend_data["growth_percentage"] = ((trend_data["growth_values"] / trend_data["baseline_subscribers"]) * 100).round(2) # Calculate percentage growth relative to the initial subscriber count

In [39]:
# Filter accounts that experienced more than 10% growth in the last month
high_growth_accounts = trend_data[trend_data["growth_percentage"] > 10.]
high_growth_accounts

,account_id,baseline_subscribers,current_subscribers,growth_values,growth_percentage
6,00374295-8694-4f4f-b73a-5d018cffd913,34405012.98,130913079.85,96508066.87,280.51
7,003bfc0f-02d3-44d3-b666-6311b6d9424b,8040.79,11666.82,3626.03,45.10
9,00450035-c58c-4dd9-86eb-9770729c4b8f,5975.76,21444.47,15468.71,258.86
11,005cf49d-1237-4124-95c3-4985dbb82d90,501202.13,605437.88,104235.75,20.80
14,006a7383-4546-4c7b-883c-a7a549995ce1,15112860.24,135498800.31,120385940.07,796.58
...,...,...,...,...,...
9861,ff8ec89e-5fba-47c5-a7ff-b70e5b610d77,271642.32,724515.40,452873.07,166.72
9866,ffacd0ac-708e-4ec3-8471-c1e385de92c1,2632.26,4931.42,2299.16,87.35
9868,ffb075ff-e587-418b-8639-920bf2948be4,4850068.67,8259765.93,3409697.26,70.30
9871,ffbb5cff-df6b-4f18-bfeb-34365a42f887,1684719.87,2823061.91,1138342.04,67.57
